# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

In [2]:
import pandas as pd

import os
import sys

sys.path.append(os.getenv('SRC_DIR'))

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [13]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [14]:
X = adult_dt.drop(columns=['income'])
Y = adult_dt['income']

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

print(X_train.head())
print(Y_train.head())

       age          workclass  fnlwgt      education  education-num  \
19749   34   Self-emp-not-inc   56460        HS-grad              9   
1216    48   Self-emp-not-inc  243631   Some-college             10   
27962   23          State-gov   56402   Some-college             10   
23077   56          Local-gov  255406        HS-grad              9   
10180   17            Private  297246           11th              7   

            marital-status        occupation    relationship  \
19749   Married-civ-spouse   Farming-fishing            Wife   
1216    Married-civ-spouse      Craft-repair         Husband   
27962   Married-civ-spouse    Prof-specialty         Husband   
23077             Divorced   Exec-managerial   Not-in-family   
10180        Never-married   Priv-house-serv       Own-child   

                      race      sex  capital-gain  capital-loss  \
19749                White   Female             0          2179   
1216    Amer-Indian-Eskimo     Male          7688     

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

The random_state parameter is used to split the dataset in train and test sets.
By assigning a value (of 42) to the random state, the model will give the same split every time it is run. In the event the code is run again later, the split will be the same, ensuring the result of the code does not change. This parameter makes the code reproducible (at a later date and by multiple users who might work with the same line of code).

*(Comment here.)*

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [28]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

numerical_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

numerical_pipeline = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

processor = ColumnTransformer(
    transformers=[('num', numerical_pipeline, numerical_cols),
                  ('cat', categorical_pipeline, categorical_cols)]
)    


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [29]:
from sklearn.ensemble import RandomForestClassifier

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, numerical_cols),
    ('cat', categorical_pipeline, categorical_cols)
])
adult_pipeline = Pipeline(steps=[('preprocessing', preprocessor), ('randomforest', RandomForestClassifier())])


# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [30]:
from sklearn.model_selection import cross_validate

scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']

results = cross_validate(adult_pipeline, X, Y, cv=5, scoring=scoring, return_train_score=True)


c:\Users\Anca\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Anca\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [32]:
results_df = pd.DataFrame(results)

sorted_results_df = results_df.sort_values(by='test_neg_log_loss', ascending=False)

print(sorted_results_df)

    fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
3  16.381573    0.297407          -0.364850           -0.080603      0.903769   
2  16.633310    0.281750          -0.373088           -0.080376      0.903255   
0  19.163057    0.315916          -0.374181           -0.080510      0.901380   
4  16.258301    0.282958          -0.387393           -0.081241      0.906075   
1  16.724360    0.280979          -0.395585           -0.080267      0.896712   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
3            1.0       0.858415        1.000000                0.777633   
2            1.0       0.851351        0.999885                0.775594   
0            1.0       0.854291        1.000000                0.772195   
4            1.0       0.856265        0.999923                0.779919   
1            1.0       0.850891        0.999923                0.770500   

   train_balanced_accuracy  
3                 1.000000  
2   

Calculate the mean of each metric. 

In [34]:
mean_results = sorted_results_df.mean()

print(f"Mean Results:\n{mean_results}")

Mean Results:
fit_time                   17.032120
score_time                  0.291802
test_neg_log_loss          -0.379019
train_neg_log_loss         -0.080600
test_roc_auc                0.902238
train_roc_auc               1.000000
test_accuracy               0.854243
train_accuracy              0.999946
test_balanced_accuracy      0.775168
train_balanced_accuracy     0.999899
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [41]:
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score, balanced_accuracy_score

adult_pipeline.fit(X_train, Y_train)

Y_pred = adult_pipeline.predict(X_test)
Y_pred_proba = adult_pipeline.predict_proba(X_test)

neg_log_loss = log_loss(Y_test, Y_pred_proba)
roc_auc = roc_auc_score(Y_test, Y_pred_proba[:, 1])
accuracy = accuracy_score(Y_test, Y_pred)
balanced_accuracy = balanced_accuracy_score(Y_test, Y_pred)

results_display = {'Negative Log Loss': neg_log_loss,
                'ROC AUC': roc_auc,
                'Accuracy': accuracy,
                'Balanced Accuracy': balanced_accuracy}

print(results_display)

{'Negative Log Loss': 0.37481750078036213, 'ROC AUC': 0.9003987335131503, 'Accuracy': 0.8558706111167981, 'Balanced Accuracy': 0.775930663207131}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

By recording the target variable at the beginning of the process, we follow best practices for machine learning. With this line of code, we proprocess the data for analysis and modeling. We clean the data by removing any white space, make it more efficient by transforming it into binary data and selecting only the answers where the value in greater than '50K'.

(Answer here.)

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.